In [1]:
import joblib
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import json
import os
import random
import pickle

random.seed(42)

# Fileter and split generated queries into seen and unseen set

In [ ]:
# filter generated queries 
file1 = open('/home/vk352/ANCE/NQ320k_dataset_v2/passages_30.json', 'r')
lines = file1.readlines()
dictq = defaultdict(set)
for line in tqdm(lines):
    item = json.loads(line)
    dictq[item['gen_question']].add(item['doc_id'])

In [ ]:
duplicates = []
for k, v in dictq.items():
    if len(v)>1:
        duplicates.append(k)

In [ ]:
duplicates = set(duplicates)
passages_id = passages_unseen_id = 0
file =  open('/home/vk352/ANCE/NQ320k_dataset_v2/passages_seen.json', 'w')
file1 = open('/home/vk352/ANCE/NQ320k_dataset_v2/passages_unseen.json', 'w')
for i in tqdm(range(0, len(lines), 30)):
    lines_block = lines[i:i+30]
    random.shuffle(lines_block)
    j = 0
    for k in range(30):
        item = json.loads(lines_block[k])
        if item['gen_question'] not in duplicates:
            item['gen_ques_id'] = passages_id
            json.dump(item, file)
            file.write('\n')
            passages_id += 1
            j += 1
        if j== 15: break
    for l in range(k+1, 30):
        item = json.loads(lines_block[l])
        if item['gen_question'] not in duplicates:
            item['gen_ques_id'] = passages_unseen_id
            json.dump(item, file1)
            file1.write('\n')
            passages_unseen_id += 1
file.close()
file1.close()

# Generate old, new and tune document splits

In [2]:
np.random.seed(0)

In [3]:
perm = np.random.permutation(109715) # there are 109715 docs in NQ

In [4]:
num_docs = 109715
percent_90 = int(0.9* num_docs)
percent_99 = int(0.99* num_docs)
old_docs = perm[:percent_90]
new_docs = perm[percent_90:percent_99]
tune_docs = perm[percent_99:]

In [5]:
len(old_docs), len(new_docs), len(tune_docs)

(98743, 9874, 1098)

In [6]:
typelist = ['testqueries', 'trainqueries', 'passages_seen', 'passages_unseen']
# typelist = ['testqueries']

for type_file in typelist:
    new_folder = '/home/vk352/dsi/data/NQ320k/'
    old_id = new_id = tune_id = 0
    file = open(f'/home/vk352/ANCE/NQ320k_dataset_v2/{type_file}.json', 'r')
    if type_file=='trainqueries':
        type_file = 'naturalqueries'
    old_docs_file = open(os.path.join(new_folder, 'old_docs', f'{type_file}.json'), 'w')
    new_docs_file = open(os.path.join(new_folder, 'new_docs', f'{type_file}.json'), 'w')
    tune_docs_file = open(os.path.join(new_folder, 'tune_docs', f'{type_file}.json'), 'w')
    
    lines = file.readlines()

    for line in tqdm(lines):
        item = json.loads(line)
        if item["doc_id"] in old_docs:
            item['ques_id'] = old_id
            json.dump(item, old_docs_file)
            old_docs_file.write('\n')
            old_id += 1
        elif item["doc_id"] in new_docs:
            item['ques_id'] = new_id
            json.dump(item, new_docs_file)
            new_docs_file.write('\n')
            new_id += 1
        elif item["doc_id"] in tune_docs:
            item['ques_id'] = tune_id
            json.dump(item, tune_docs_file)
            tune_docs_file.write('\n')
            tune_id += 1

    print(f"Type of file: {type_file}")
    print(f"Number in Old Docs: {old_id}")
    print(f"Number in New Docs: {new_id}")
    print(f"Number in Tune Docs: {tune_id}")
    print(f"Total: {old_id+new_id+tune_id}")
    
    
    old_docs_file.close()
    new_docs_file.close()
    tune_docs_file.close()




100%|██████████| 7830/7830 [00:00<00:00, 17794.71it/s]


Type of file: testqueries
Number in Old Docs: 6998
Number in New Docs: 738
Number in Tune Docs: 94
Total: 7830


100%|██████████| 307373/307373 [00:15<00:00, 19861.95it/s]


Type of file: naturalqueries
Number in Old Docs: 276493
Number in New Docs: 27723
Number in Tune Docs: 3157
Total: 307373


100%|██████████| 1645085/1645085 [01:23<00:00, 19698.73it/s]


Type of file: passages_seen
Number in Old Docs: 1480538
Number in New Docs: 148077
Number in Tune Docs: 16470
Total: 1645085


100%|██████████| 1471338/1471338 [01:14<00:00, 19754.31it/s]


Type of file: passages_unseen
Number in Old Docs: 1324114
Number in New Docs: 132415
Number in Tune Docs: 14809
Total: 1471338


In [7]:
# split train queries into train and val set using a 80-20 split
new_folder = '/home/vk352/dsi/data/NQ320k/'
folders = ['old_docs', 'new_docs', 'tune_docs']
for folder in folders:
    docs_file = open(os.path.join(new_folder, folder, f'naturalqueries.json'), 'r')
    docs_file_train = open(os.path.join(new_folder, folder, f'trainqueries.json'), 'w')
    docs_file_val = open(os.path.join(new_folder, folder, f'valqueries.json'), 'w')
    
    train_id = val_id = 0
    
    lines = docs_file.readlines()
    perm_split = np.random.permutation(len(lines))
    split = int(0.8*len(lines))
    for index in tqdm(perm_split[:split]):
        item = json.loads(lines[index])
        item['ques_id'] = train_id
        json.dump(item, docs_file_train)
        docs_file_train.write('\n')
        train_id += 1
        
    for index in tqdm(perm_split[split:]):
        item = json.loads(lines[index])
        item['ques_id'] = val_id
        json.dump(item, docs_file_val)
        docs_file_val.write('\n')
        val_id += 1

    print(f"Type of folder: {folder}")
    print(f"Number in Train: {train_id}")
    print(f"Number in Val: {val_id}")
    print(f"Total: {train_id+val_id}")


    docs_file.close()
    docs_file_train.close()
    docs_file_val.close()

100%|██████████| 55299/55299 [00:00<00:00, 89953.90it/s]


Type of folder: old_docs
Number in Train: 221194
Number in Val: 55299
Total: 276493


100%|██████████| 5545/5545 [00:00<00:00, 89789.00it/s]


Type of folder: new_docs
Number in Train: 22178
Number in Val: 5545
Total: 27723


100%|██████████| 632/632 [00:00<00:00, 56557.64it/s]

Type of folder: tune_docs
Number in Train: 2525
Number in Val: 632
Total: 3157


# write mappings for ANCE/DPR

In [8]:
# write mapping
docid2quesid = {}
quesid2docid = {}

file1 = open('/home/vk352/dsi/data/NQ320k/old_docs/passages_seen.json', 'r')
lines = file1.readlines()
for line in lines:
    item = json.loads(line)
    quesid2docid[item['gen_ques_id']] = item['doc_id']
    if item['doc_id'] not in docid2quesid:
        docid2quesid[item['doc_id']] = [item['gen_ques_id']]
    else:
        docid2quesid[item['doc_id']].append(item['gen_ques_id'])
        

with open('/home/vk352/dsi/data/NQ320k/old_docs/quesid2docid.pkl', 'wb') as f:
    pickle.dump(quesid2docid, f)
with open('/home/vk352/dsi/data/NQ320k/old_docs/docid2quesid.pkl', 'wb') as f:
    pickle.dump(docid2quesid, f)